# 1.输入与输出（使用自定义）

In [2]:
# 多输入通道互相关运算
import torch
from d2l import torch as d2l
from torch import nn

# 多通道输入运算
def corr2d_multi_in(X,K):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K)) # X,K为3通道矩阵，for使得对最外面通道进行遍历        

X = torch.tensor([[[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]]
                  ,[[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]
                 ])
K = torch.tensor([[[0.0,1.0],[2.0,3.0]]
                  ,[[1.0,2.0],[3.0,4.0]]
                 ])

# print('X.shape:x\n',X.shape)
print('X:\n',X)
# print('K.shape:x\n',K.shape)
print('X:\n',K)
print(corr2d_multi_in(X,K))
print(corr2d_multi_in(X,K+1))
print(corr2d_multi_in(X,K+2))
print('-----------')
# 多输出通道运算
def corr2d_multi_in_out(X,K):  # X为3通道矩阵，K为4通道矩阵，最外维为输出通道      
    return torch.stack([corr2d_multi_in(X,k) for k in K],0) # 大k中每个小k是一个3D的Tensor。0表示stack堆叠函数里面在0这个维度堆叠。           
    
# print('X.shape:',X.shape)
# print('K.shape:',K.shape)
# print(K)
# print(K+1)
# print(K+2)
K = torch.stack((K, K+1, K+2),0) # K与K+1之间的区别为K的每个元素加1
print(K.shape)
print(corr2d_multi_in_out(X,K))

X:
 tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])
X:
 tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])
tensor([[ 56.,  72.],
        [104., 120.]])
tensor([[ 76., 100.],
        [148., 172.]])
tensor([[ 96., 128.],
        [192., 224.]])
-----------
torch.Size([3, 2, 2, 2])
tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


# 2.1x1卷积（使用矩阵乘法实现 等价于全连接层）

In [4]:
# 1×1卷积的多输入、多输出通道运算
def corr2d_multi_in_out_1x1(X,K):
    c_i, h, w = X.shape # 输入的通道数、宽、高
    c_o = K.shape[0]    # 输出的通道数
    X = X.reshape((c_i, h * w)) # 拉平操作，每一行表示一个通道的特征
    K = K.reshape((c_o,c_i)) 
    Y = torch.matmul(K,X) 
    return Y.reshape((c_o, h, w))

X = torch.normal(0,1,(3,3,3))   # norm函数生成0到1之间的(3,3,3)矩阵 
K = torch.normal(0,1,(2,3,1,1)) # 输出通道是2，输入通道是3，核是1X1

Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
print('X:\n',X.shape,'\n',X)
print('K:\n',K.shape,'\n',K)
print('Y1:\n',Y1.shape,'\n',Y1)
print('Y2:\n',Y2.shape,'\n',Y2)
assert float(torch.abs(Y1-Y2).sum()) < 1e-6

X:
 torch.Size([3, 3, 3]) 
 tensor([[[-0.3385,  0.1138, -0.7406],
         [-0.7188,  1.9767,  0.4188],
         [-2.7680,  0.3167,  0.3663]],

        [[-1.1692, -0.9002,  0.2707],
         [-0.7538,  2.8419,  0.8217],
         [-0.4371,  0.3489, -0.6437]],

        [[ 0.0192, -1.6957, -0.0590],
         [-0.5724, -0.2601,  0.5234],
         [-0.0325, -0.6103,  0.3157]]])
K:
 torch.Size([2, 3, 1, 1]) 
 tensor([[[[ 0.1360]],

         [[ 0.1583]],

         [[-0.1457]]],


        [[[ 0.7564]],

         [[ 0.2525]],

         [[ 0.6310]]]])
Y1:
 torch.Size([2, 3, 3]) 
 tensor([[[-0.2339,  0.1201, -0.0493],
         [-0.1336,  0.7565,  0.1107],
         [-0.4409,  0.1872, -0.0980]],

        [[-0.5392, -1.2113, -0.5291],
         [-1.0953,  2.0488,  0.8545],
         [-2.2246, -0.0574,  0.3138]]])
Y2:
 torch.Size([2, 3, 3]) 
 tensor([[[-0.2339,  0.1201, -0.0493],
         [-0.1336,  0.7565,  0.1107],
         [-0.4409,  0.1872, -0.0980]],

        [[-0.5392, -1.2113, -0.5291],
        